<a href="https://colab.research.google.com/github/Rainniee/Neural-Networks-AI/blob/master/Experiment%204_H2O%20Auto%20ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import division
from __future__ import print_function

import pandas as pd
import numpy as np

import itertools
import os

import nltk
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix, accuracy_score

from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder


%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.datasets import imdb
from tensorflow import keras
from keras.layers import Embedding, Flatten, Dense, SimpleRNN, LSTM, Activation, Dropout
from keras.layers import LSTM, CuDNNLSTM

from keras.models import Sequential
from keras.models import load_model
from keras.preprocessing import text, sequence
from keras import utils
from tensorflow.keras.callbacks import EarlyStopping

# This code was tested with TensorFlow v1.4
print("You have TensorFlow version", tf.__version__)

from distutils.version import LooseVersion as LV
from keras import __version__
from keras import backend as K

import seaborn as sns
sns.set()

Using TensorFlow backend.


You have TensorFlow version 1.13.1


In [2]:
!pip install h2o

    100% |████████████████████████████████| 121.0MB 218kB/s 
  Stored in directory: /root/.cache/pip/wheels/ca/0a/e6/f0538dc7e67f9c76b5c93dcc3529c1a00adeed4dacd31d551a
Successfully built h2o


In [3]:
import h2o
h2o.init()
from h2o.estimators.word2vec import H2OWord2vecEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "10.0.2" 2018-07-17; OpenJDK Runtime Environment (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4); OpenJDK 64-Bit Server VM (build 10.0.2+13-Ubuntu-1ubuntu0.18.04.4, mixed mode)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpmscgzjc_
  JVM stdout: /tmp/tmpmscgzjc_/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpmscgzjc_/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,1 day
H2O cluster name:,H2O_from_python_unknownUser_dvad9d
H2O cluster total nodes:,1
H2O cluster free memory:,2.938 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [4]:
from google.colab import files
upload = files.upload()

Saving merged_api_score.csv to merged_api_score (1).csv


In [5]:
h2o_data = h2o.import_file("merged_api_score.csv")
h2o_data.head()

Parse progress: |█████████████████████████████████████████████████████████| 100%


azure,google,ibm,label
0.815189,0.2,0,2
0.15845,0,0,1
0.5,0,-0.183405,1
0.758991,0.2,0.554778,2
0.142382,0,0,0
0.5,0,0.82987,2
0.803309,0,0,2
0.910819,0.2,0,0
0.183081,0.1,0,0
0.5,0.4,0.38213,0


In [0]:
train,test,valid= h2o_data.split_frame(ratios=[.7, .15])

In [7]:
# Train the model
from h2o.automl import H2OAutoML

aml = H2OAutoML(max_models=30, seed=99,max_runtime_secs =6000)

x = train.columns
y = "label"
x.remove(y)

train[y] = train[y].asfactor()
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [8]:
# View the AutoML Leaderboard
lb = aml.leaderboard
print (lb)

model_id,mean_per_class_error,logloss,rmse,mse
DeepLearning_grid_1_AutoML_20190402_180520_model_5,0.551064,0.900581,0.564019,0.318118
StackedEnsemble_BestOfFamily_AutoML_20190402_180520,0.552608,0.86372,0.553188,0.306017
DeepLearning_grid_1_AutoML_20190402_180520_model_3,0.55383,0.951389,0.555784,0.308896
GBM_grid_1_AutoML_20190402_180520_model_2,0.554632,0.870487,0.553314,0.306157
DeepLearning_grid_1_AutoML_20190402_180520_model_2,0.554921,0.9314,0.552502,0.305259
GBM_grid_1_AutoML_20190402_180520_model_1,0.556476,1.08477,0.66199,0.43823
DeepLearning_grid_1_AutoML_20190402_180520_model_4,0.557485,0.899835,0.555174,0.308219
XRT_1_AutoML_20190402_180520,0.558329,1.0895,0.557862,0.31121
DeepLearning_grid_1_AutoML_20190402_180520_model_1,0.558538,0.947142,0.566306,0.320703
GBM_1_AutoML_20190402_180520,0.55995,0.869403,0.557416,0.310713


In [9]:
# Predict
test_y = aml.leader.predict(test)

deeplearning prediction progress: |███████████████████████████████████████| 100%


In [10]:
h2o.export_file(test_y,"preds.csv") 
h2o.export_file(test,"test_res.csv") 

preds = pd.read_csv('preds.csv') 
test = pd.read_csv('test_res.csv') 
test = test.fillna(0)
preds = preds['predict'].astype(float)

from sklearn.metrics import accuracy_score

print("H2O Auto ML Accuracy Score is "+str(accuracy_score(preds,test['label'])))

Export File progress: |███████████████████████████████████████████████████| 100%
Export File progress: |███████████████████████████████████████████████████| 100%
H2O Auto ML Accuracy Score is 0.6
